In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import datetime as dt

np.random.seed(123)

In [106]:
def get_data(tickers, start, end):
    closes = yf.download(tickers, start=start, end=end, progress=False)['Adj Close']
    rt = closes.pct_change().dropna()
    mean, cov_m = rt.mean(), rt.cov()
    return mean, cov_m, closes

In [107]:
def simular_precios(mean_rt, matrix_cov, num_dias, stocks, weights):
    
    N = 1000

    portfolio_return = mean_rt.dot(weights)
    
    portfolio_simulated_returns = np.full(shape = (num_dias, N), fill_value=0)
    mean_rt = np.full(shape=(num_dias, len(stocks)), fill_value=portfolio_return)
    
    for n in range(N):
        L = np.linalg.cholesky(matrix_cov)
        Z = np.random.normal(size=(num_dias, len(stocks))) # Matriz de n_dias (filas), num_activos (columnas)

        daily_returns = mean_rt.T + L.dot(Z.T)
        portfolio_simulated_returns[:, n] = np.cumprod( np.dot(weights, daily_returns) + 1)
        
        
    print('portfolio', portfolio_simulated_returns)

In [108]:
tickers = ['AAPL', 'MSFT', 'TSLA', 'F']
weights = np.ones(len(tickers)) / len(tickers)

mean_rt, cov_m, closes = get_data(tickers, dt.datetime.now() - dt.timedelta(days=365*3), dt.datetime.now())

In [109]:
simular_precios(mean_rt, cov_m, 7, tickers, weights)

portfolio [[0 0 1 ... 0 1 1]
 [0 0 1 ... 0 1 0]
 [0 0 1 ... 0 1 0]
 ...
 [1 0 1 ... 0 1 0]
 [1 0 1 ... 0 1 1]
 [1 0 1 ... 0 1 0]]
